In [1]:
#Dependencies
import pandas as pd
from tswift import Song
import requests
from lxml import html
import os

import string

In [2]:
#read 2016-2017 csv

data_20162017 = pd.read_csv("billboard_lyrics_2016-2017.csv",encoding="mbcs")

print(data_20162017.head())
print(len(data_20162017["Title"]))


   No.            Title                        Artist(s)  Year
0    1  "Love Yourself"                    Justin Bieber  2016
1    2          "Sorry"                    Justin Bieber  2016
2    3      "One Dance"  Drake featuring Wizkid and Kyla  2016
3    4           "Work"          Rihanna featuring Drake  2016
4    5   "Stressed Out"                Twenty One Pilots  2016
200


In [3]:
#Data Cleanup

#rename column No. and Artist(s)
data_20162017 = data_20162017.rename(columns={"No.":"Rank","Artist(s)":"Artist"})

#remove quotes/apostrophes/punctuation from title
data_20162017 = data_20162017.replace(to_replace=['"',"'",",","&","-","\?"],value=["","","","","",""],regex=True)

#remove accents from artist
data_20162017 = data_20162017.replace(to_replace=["ó","é","\."],value=["o","e",""],regex=True)

#set strings to lower case
data_20162017["Title"] = data_20162017["Title"].str.lower()
data_20162017["Artist"] = data_20162017["Artist"].str.lower()

print(data_20162017.head())
print(len(data_20162017["Title"]))

   Rank          Title                           Artist  Year
0     1  love yourself                    justin bieber  2016
1     2          sorry                    justin bieber  2016
2     3      one dance  drake featuring wizkid and kyla  2016
3     4           work          rihanna featuring drake  2016
4     5   stressed out                twenty one pilots  2016
200


In [4]:
#create a blank lyrics column in the 20162017 dataframe
data_20162017 = data_20162017.assign(Lyrics="",Source="",CheckURL="")

print(data_20162017.head())

   Rank          Title                           Artist  Year CheckURL Lyrics  \
0     1  love yourself                    justin bieber  2016                   
1     2          sorry                    justin bieber  2016                   
2     3      one dance  drake featuring wizkid and kyla  2016                   
3     4           work          rihanna featuring drake  2016                   
4     5   stressed out                twenty one pilots  2016                   

  Source  
0         
1         
2         
3         
4         


In [5]:
#read 1964-2015 csv
data_19642015 = pd.read_csv("billboard_lyrics_1964-2015.csv",encoding="mbcs")

#rename column Song to Title
data_19642015 = data_19642015.rename(columns={"Song":"Title"})
data_19642015 = data_19642015.replace(to_replace=["  ","NaN"],value=[", ",""],regex=True)

data_19642015 = data_19642015.assign(CheckURL="")

data_19642015.head()

,Rank,Title,Artist,Year,Lyrics,Source,CheckURL
0,1,wooly bully,sam the sham and the pharaohs,1965,sam the sham miscellaneous wooly bully wooly b...,3.0,
1,2,i cant help myself sugar pie honey bunch,four tops,1965,sugar pie honey bunch you know that i love yo...,1.0,
2,3,i cant get no satisfaction,the rolling stones,1965,",",1.0,
3,4,you were on my mind,we five,1965,when i woke up this morning you were on my mi...,1.0,
4,5,youve lost that lovin feelin,the righteous brothers,1965,you never close your eyes anymore when i kiss...,1.0,


In [6]:
#append 20162017 to 19642015 dataframe
data_new = data_19642015.append(data_20162017)

data_new = data_new.reset_index()

data_new = data_new.drop("index",axis=1)

data_new.head()

,Artist,CheckURL,Lyrics,Rank,Source,Title,Year
0,sam the sham and the pharaohs,,sam the sham miscellaneous wooly bully wooly b...,1,3,wooly bully,1965
1,four tops,,sugar pie honey bunch you know that i love yo...,2,1,i cant help myself sugar pie honey bunch,1965
2,the rolling stones,,",",3,1,i cant get no satisfaction,1965
3,we five,,when i woke up this morning you were on my mi...,4,1,you were on my mind,1965
4,the righteous brothers,,you never close your eyes anymore when i kiss...,5,1,youve lost that lovin feelin,1965


In [7]:
#find lyrics from metrolyrics
check_url = ""
for index, item in enumerate(data_new["Title"]):
    #create url to be passed in metrolyrics

    try:
        #if index == 3:
        #    break
        song_url = "http://www.metrolyrics.com/"
        
        #parse title and append it to the base url
        parsed_title = item.split()
        
        for i in parsed_title:

            song_url = song_url + i + "-"

        song_url = song_url + "lyrics-"
        title_url = song_url
        #parse artist and append it to the base url (including only the first artist on the song)
        artist = data_new["Artist"].get(index)
        
        parsed_artist  = artist.split()
        
        #only check with the main artists name (without featured artists)
        for i in parsed_artist:
            
            if i == "featuring" or i == "and" or i == "&" or i=="with":    
                
                break            
            #print(parsed_artist)
            if i != i.strip(","):
                
                i = i.strip(",")
                song_url = song_url + i + "-"
                
                break
            i= i.strip(".")    
            song_url = song_url + i + "-"
            
        
        song_url = song_url[:-1] +".html"
        
        #show which song is being checked on
        print("Checking #" + str(index) + " URL: " + str(song_url))
        data_new.set_value(index,"CheckURL",song_url)
        
        #get lyrics from song_url
        page = requests.get(song_url)
    
        #Forces utf-8 to prevent character mangling

        page.encoding = 'utf-8'

        tree = html.fromstring(page.text)
        lyric_div = tree.get_element_by_id('lyrics-body-text')
        verse = [c.text_content() for c in lyric_div.find_class('verse')]
        
        #print(parsed_artist)
        if verse[0] == "":

            raise ValueError
        #remove newline from lyrics
        verse = [i.replace("\n"," ") for i in verse]
        
        #concatenate items in the lyrics list to string
        a = ""
        for i in verse:
            a = a + " " + i
        verse = a
        verse = verse[1:]
        #print(verse)
        #set lyrics and source values in the dataframe
        if verse !="":
            data_new.set_value(index,"Lyrics",verse)
        data_new.set_value(index,"Source",1)        
        

    except:
        
        try:
            #recheck metrolyrics without "the" if it starts with "the"
            if parsed_artist[0] == "the":

                del parsed_artist[0]

                for i in parsed_artist:
                      
                    title_url = title_url + i + "-"
                    
                song_url = title_url[:-1] +".html"

                #show which song is being checked on
                print("Rechecking #" + str(index) + " URL: " + str(song_url))
                
                #data_new.set_value(index,"CheckURL",song_url)
                
                #get lyrics from song_url
                page = requests.get(song_url)
                
                #Forces utf-8 to prevent character mangling

                page.encoding = 'utf-8'

                tree = html.fromstring(page.text)
                lyric_div = tree.get_element_by_id('lyrics-body-text')
                verse = [c.text_content() for c in lyric_div.find_class('verse')]
                
                #remove newline from lyrics
                verse = [i.replace("\n"," ") for i in verse]
                
                
                #concatenate items in the lyrics list to string
                a = ""
                for i in verse:
                    a = a + " " + i
                verse = a
                verse = verse[1:]
                
                #set lyrics and source values in the dataframe
                if verse !="":
                    data_new.set_value(index,"Lyrics",verse)
                data_new.set_value(index,"Source",1)        

        except:
                    
                next

        
        next

print(data_new.head())

Checking #0 URL: http://www.metrolyrics.com/wooly-bully-lyrics-sam-the-sham.html
Checking #1 URL: http://www.metrolyrics.com/i-cant-help-myself-sugar-pie-honey-bunch-lyrics-four-tops.html
Checking #2 URL: http://www.metrolyrics.com/i-cant-get-no-satisfaction-lyrics-the-rolling-stones.html
Rechecking #2 URL: http://www.metrolyrics.com/i-cant-get-no-satisfaction-lyrics-rolling-stones.html
Checking #3 URL: http://www.metrolyrics.com/you-were-on-my-mind-lyrics-we-five.html
Checking #4 URL: http://www.metrolyrics.com/youve-lost-that-lovin-feelin-lyrics-the-righteous-brothers.html
Checking #5 URL: http://www.metrolyrics.com/downtown-lyrics-petula-clark.html
Checking #6 URL: http://www.metrolyrics.com/help-lyrics-the-beatles.html
Rechecking #6 URL: http://www.metrolyrics.com/help-lyrics-beatles.html
Checking #7 URL: http://www.metrolyrics.com/cant-you-hear-my-heart-beat-lyrics-hermans-hermits.html
Checking #8 URL: http://www.metrolyrics.com/crying-in-the-chapel-lyrics-elvis-presley.html
Check

Checking #75 URL: http://www.metrolyrics.com/red-roses-for-a-blue-lady-lyrics-vic-dana.html
Checking #76 URL: http://www.metrolyrics.com/dont-just-stand-there-lyrics-patty-duke.html
Checking #77 URL: http://www.metrolyrics.com/the-tracks-of-my-tears-lyrics-the-miracles.html
Rechecking #77 URL: http://www.metrolyrics.com/the-tracks-of-my-tears-lyrics-miracles.html
Checking #78 URL: http://www.metrolyrics.com/too-many-rivers-lyrics-brenda-lee.html
Checking #79 URL: http://www.metrolyrics.com/i-like-it-like-that-lyrics-the-dave-clark-five.html
Rechecking #79 URL: http://www.metrolyrics.com/i-like-it-like-that-lyrics-dave-clark-five.html
Checking #80 URL: http://www.metrolyrics.com/little-things-lyrics-bobby-goldsboro.html
Checking #81 URL: http://www.metrolyrics.com/true-love-ways-lyrics-peter.html
Checking #82 URL: http://www.metrolyrics.com/its-the-same-old-song-lyrics-four-tops.html
Checking #83 URL: http://www.metrolyrics.com/youve-got-your-troubles-lyrics-the-fortunes.html
Rechecking

Rechecking #140 URL: http://www.metrolyrics.com/daydream-lyrics-lovin-spoonful.html
Checking #141 URL: http://www.metrolyrics.com/time-wont-let-me-lyrics-the-outsiders.html
Rechecking #141 URL: http://www.metrolyrics.com/time-wont-let-me-lyrics-outsiders.html
Checking #142 URL: http://www.metrolyrics.com/hooray-for-hazel-lyrics-tommy-roe.html
Checking #143 URL: http://www.metrolyrics.com/sweet-pea-lyrics-tommy-roe.html
Checking #144 URL: http://www.metrolyrics.com/bus-stop-lyrics-the-hollies.html
Rechecking #144 URL: http://www.metrolyrics.com/bus-stop-lyrics-hollies.html
Checking #145 URL: http://www.metrolyrics.com/im-your-puppet-lyrics-james.html
Checking #146 URL: http://www.metrolyrics.com/im-so-lonesome-i-could-cry-lyrics-bj-thomas.html
Checking #147 URL: http://www.metrolyrics.com/aint-too-proud-to-beg-lyrics-the-temptations.html
Rechecking #147 URL: http://www.metrolyrics.com/aint-too-proud-to-beg-lyrics-temptations.html
Checking #148 URL: http://www.metrolyrics.com/dirty-water

Checking #215 URL: http://www.metrolyrics.com/kind-of-a-drag-lyrics-the-buckinghams.html
Rechecking #215 URL: http://www.metrolyrics.com/kind-of-a-drag-lyrics-buckinghams.html
Checking #216 URL: http://www.metrolyrics.com/sweet-soul-music-lyrics-arthur-conley.html
Checking #217 URL: http://www.metrolyrics.com/expressway-to-your-heart-lyrics-the-soul-survivors.html
Rechecking #217 URL: http://www.metrolyrics.com/expressway-to-your-heart-lyrics-soul-survivors.html
Checking #218 URL: http://www.metrolyrics.com/soul-man-lyrics-sam.html
Checking #219 URL: http://www.metrolyrics.com/never-my-love-lyrics-the-association.html
Rechecking #219 URL: http://www.metrolyrics.com/never-my-love-lyrics-association.html
Checking #220 URL: http://www.metrolyrics.com/apples-peaches-pumpkin-pie-lyrics-jay.html
Checking #221 URL: http://www.metrolyrics.com/come-on-down-to-my-boat-lyrics-every-mothers-son.html
Checking #222 URL: http://www.metrolyrics.com/incense-and-peppermints-lyrics-strawberry-alarm-clock

Checking #280 URL: http://www.metrolyrics.com/white-rabbit-lyrics-jefferson-airplane.html
Checking #281 URL: http://www.metrolyrics.com/bernadette-lyrics-four-tops.html
Checking #282 URL: http://www.metrolyrics.com/the-beat-goes-on-lyrics-sonny.html
Checking #283 URL: http://www.metrolyrics.com/snoopy-vs-the-red-baron-lyrics-the-royal-guardsmen.html
Rechecking #283 URL: http://www.metrolyrics.com/snoopy-vs-the-red-baron-lyrics-royal-guardsmen.html
Checking #284 URL: http://www.metrolyrics.com/societys-child-lyrics-janis-ian.html
Checking #285 URL: http://www.metrolyrics.com/girl-youll-be-a-woman-soon-lyrics-neil-diamond.html
Checking #286 URL: http://www.metrolyrics.com/aint-no-mountain-high-enough-lyrics-marvin-gaye.html
Checking #287 URL: http://www.metrolyrics.com/i-take-it-back-lyrics-sandy-posey.html
Checking #288 URL: http://www.metrolyrics.com/here-comes-my-baby-lyrics-the-tremeloes.html
Rechecking #288 URL: http://www.metrolyrics.com/here-comes-my-baby-lyrics-tremeloes.html
Che

...

Checking #4827 URL: http://www.metrolyrics.com/i-love-it-lyrics-icona-pop.html
Checking #4828 URL: http://www.metrolyrics.com/safe-and-sound-lyrics-capital-cities.html
Checking #4829 URL: http://www.metrolyrics.com/treasure-lyrics-bruno-mars.html
Checking #4830 URL: http://www.metrolyrics.com/the-way-lyrics-ariana-grande.html
Checking #4831 URL: http://www.metrolyrics.com/started-from-the-bottom-lyrics-drake.html
Checking #4832 URL: http://www.metrolyrics.com/come,-get-it-lyrics-selena-gomez.html
Checking #4833 URL: http://www.metrolyrics.com/hold-on-were-going-home-lyrics-drake.html
Checking #4834 URL: http://www.metrolyrics.com/daylight-lyrics-maroon-5.html
Checking #4835 URL: http://www.metrolyrics.com/feel-this-moment-lyrics-pitbull.html
Checking #4836 URL: http://www.metrolyrics.com/applause-lyrics-lady-gaga.html
Checking #4837 URL: http://www.metrolyrics.com/one-more-night-lyrics-maroon-5.html
Checking #4838 URL: http://www.metrolyrics.com/love-me-lyrics-lil-wayne.html
Checking #

Checking #4927 URL: http://www.metrolyrics.com/latch-lyrics-disclosure.html
Checking #4928 URL: http://www.metrolyrics.com/maps-lyrics-maroon-5.html
Checking #4929 URL: http://www.metrolyrics.com/loyal-lyrics-chris-brown.html
Checking #4930 URL: http://www.metrolyrics.com/best-day-of-my-life-lyrics-american-authors.html
Checking #4931 URL: http://www.metrolyrics.com/habits-stay-high-lyrics-tove-lo.html
Checking #4932 URL: http://www.metrolyrics.com/summer-lyrics-calvin-harris.html
Checking #4933 URL: http://www.metrolyrics.com/boom-clap-lyrics-charli-xcx.html
Checking #4934 URL: http://www.metrolyrics.com/drunk-in-love-lyrics-beyonce.html
Checking #4935 URL: http://www.metrolyrics.com/anaconda-lyrics-nicki-minaj.html
Checking #4936 URL: http://www.metrolyrics.com/break-free-lyrics-ariana-grande.html
Checking #4937 URL: http://www.metrolyrics.com/bailando-lyrics-enrique-iglesias.html
Checking #4938 URL: http://www.metrolyrics.com/burn-lyrics-ellie-goulding.html
Checking #4939 URL: http:

Checking #5029 URL: http://www.metrolyrics.com/hotline-bling-lyrics-drake.html
Checking #5030 URL: http://www.metrolyrics.com/hey-mama-lyrics-david-guetta.html
Checking #5031 URL: http://www.metrolyrics.com/gdfr-lyrics-flo-rida.html
Checking #5032 URL: http://www.metrolyrics.com/what-do-you-mean-lyrics-justin-bieber.html
Checking #5033 URL: http://www.metrolyrics.com/photograph-lyrics-ed-sheeran.html
Checking #5034 URL: http://www.metrolyrics.com/hello-lyrics-adele.html
Checking #5035 URL: http://www.metrolyrics.com/stitches-lyrics-shawn-mendes.html
Checking #5036 URL: http://www.metrolyrics.com/talking-body-lyrics-tove-lo.html
Checking #5037 URL: http://www.metrolyrics.com/jealous-lyrics-nick-jonas.html
Checking #5038 URL: http://www.metrolyrics.com/time-of-our-lives-lyrics-pitbull.html
Checking #5039 URL: http://www.metrolyrics.com/locked-away-lyrics-r-city.html
Checking #5040 URL: http://www.metrolyrics.com/somebody-lyrics-natalie-la-rose.html
Checking #5041 URL: http://www.metrolyr

Checking #5129 URL: http://www.metrolyrics.com/low-life-lyrics-future.html
Checking #5130 URL: http://www.metrolyrics.com/what-do-you-mean-lyrics-justin-bieber.html
Checking #5131 URL: http://www.metrolyrics.com/the-hills-lyrics-the-weeknd.html
Checking #5132 URL: http://www.metrolyrics.com/just-like-fire-lyrics-pink.html
Checking #5133 URL: http://www.metrolyrics.com/broccoli-lyrics-dram.html
Checking #5134 URL: http://www.metrolyrics.com/dont-lyrics-bryson-tiller.html
Checking #5135 URL: http://www.metrolyrics.com/dangerous-woman-lyrics-ariana-grande.html
Checking #5136 URL: http://www.metrolyrics.com/jumpman-lyrics-drake.html
Checking #5137 URL: http://www.metrolyrics.com/i-hate-u-i-love-u-lyrics-gnash.html
Checking #5138 URL: http://www.metrolyrics.com/here-lyrics-alessia-cara.html
Checking #5139 URL: http://www.metrolyrics.com/same-old-love-lyrics-selena-gomez.html
Checking #5140 URL: http://www.metrolyrics.com/controlla-lyrics-drake.html
Checking #5141 URL: http://www.metrolyrics

Checking #5231 URL: http://www.metrolyrics.com/slow-hands-lyrics-niall-horan.html
Checking #5232 URL: http://www.metrolyrics.com/love-on-the-brain-lyrics-rihanna.html
Checking #5233 URL: http://www.metrolyrics.com/i-feel-it-coming-lyrics-the-weeknd.html
Checking #5234 URL: http://www.metrolyrics.com/bounce-back-lyrics-big-sean.html
Checking #5235 URL: http://www.metrolyrics.com/strip-that-down-lyrics-liam-payne.html
Checking #5236 URL: http://www.metrolyrics.com/fake-love-lyrics-drake.html
Checking #5237 URL: http://www.metrolyrics.com/dont-want-to-know-lyrics-maroon-5.html
Checking #5238 URL: http://www.metrolyrics.com/look-what-you-made-me-do-lyrics-taylor-swift.html
Checking #5239 URL: http://www.metrolyrics.com/castle-on-the-hill-lyrics-ed-sheeran.html
Checking #5240 URL: http://www.metrolyrics.com/bad-things-lyrics-machine-gun-kelly.html
Checking #5241 URL: http://www.metrolyrics.com/paris-lyrics-the-chainsmokers.html
Checking #5242 URL: http://www.metrolyrics.com/side-to-side-lyr

In [8]:
#write output to csv

data_new.to_csv(path_or_buf=os.path.join("..","data_output.csv"))

In [9]:
data_new.head()

,Artist,CheckURL,Lyrics,Rank,Source,Title,Year
0,sam the sham and the pharaohs,http://www.metrolyrics.com/wooly-bully-lyrics-...,sam the sham miscellaneous wooly bully wooly b...,1,3,wooly bully,1965
1,four tops,http://www.metrolyrics.com/i-cant-help-myself-...,"Ooh, sugar pie, honey bunch You know that I lo...",2,1,i cant help myself sugar pie honey bunch,1965
2,the rolling stones,http://www.metrolyrics.com/i-cant-get-no-satis...,"I can't get no satisfaction, I can't get no sa...",3,1,i cant get no satisfaction,1965
3,we five,http://www.metrolyrics.com/you-were-on-my-mind...,Verse 1 When I woke up this morning You were o...,4,1,you were on my mind,1965
4,the righteous brothers,http://www.metrolyrics.com/youve-lost-that-lov...,You never close your eyes anymore when I kiss ...,5,1,youve lost that lovin feelin,1965
